# NYC Taxi Data Analysis

This notebook contains the required analytical queries for the NYC Taxi data project.

## Required Analyses:
1. Calculate average total_amount per month considering all trips
2. Calculate average passenger_count per hour in May

## Prerequisites:
- ETL pipeline must be completed (Bronze, Silver, Gold layers)
- Gold layer tables must be available in the metastore


In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
import json

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully")


In [ ]:
# Initialize Spark session (if not already available)
# In Databricks, spark is already available
try:
    spark
    print("✅ Using existing Spark session")
except NameError:
    spark = SparkSession.builder \
        .appName("NYC_Taxi_Analysis") \
        .config("spark.sql.adaptive.enabled", "true") \
        .enableHiveSupport() \
        .getOrCreate()
    print("✅ Spark session created")

print(f"Spark version: {spark.version}")


In [ ]:
# Query 1: Monthly average total_amount
monthly_query = """
SELECT 
    pickup_month,
    CASE pickup_month
        WHEN 1 THEN 'January'
        WHEN 2 THEN 'February' 
        WHEN 3 THEN 'March'
        WHEN 4 THEN 'April'
        WHEN 5 THEN 'May'
    END as month_name,
    ROUND(avg_total_amount, 2) as avg_total_amount,
    total_trips
FROM gold_monthly_aggregations 
ORDER BY pickup_month
"""

monthly_df = spark.sql(monthly_query)
monthly_pandas = monthly_df.toPandas()

print("📊 REQUIRED ANALYSIS 1: Monthly Average Total Amount")
print("=" * 55)
for _, row in monthly_pandas.iterrows():
    print(f"{row['month_name']:>9}: ${row['avg_total_amount']:>6.2f} (from {row['total_trips']:>8,} trips)")

# Display as Spark DataFrame
monthly_df.show()


In [ ]:
# Query 2: Hourly average passenger_count in May
hourly_query = """
SELECT 
    pickup_hour,
    CASE 
        WHEN pickup_hour = 0 THEN '12:00 AM'
        WHEN pickup_hour < 12 THEN CONCAT(pickup_hour, ':00 AM')
        WHEN pickup_hour = 12 THEN '12:00 PM'
        ELSE CONCAT(pickup_hour - 12, ':00 PM')
    END as hour_display,
    ROUND(avg_passenger_count, 2) as avg_passenger_count,
    total_trips
FROM gold_hourly_aggregations_may 
ORDER BY pickup_hour
"""

hourly_df = spark.sql(hourly_query)
hourly_pandas = hourly_df.toPandas()

print("📊 REQUIRED ANALYSIS 2: Hourly Average Passenger Count (May 2023)")
print("=" * 70)

# Display key hours
for i in range(0, len(hourly_pandas), 3):
    row = hourly_pandas.iloc[i]
    print(f"{row['hour_display']:>8}: {row['avg_passenger_count']:>4.2f} passengers (from {row['total_trips']:>6,} trips)")

print("\n(Full 24-hour data available in the DataFrame below)")
hourly_df.show(24)
